<a href="https://colab.research.google.com/github/rohitrawate/Engineering/blob/main/Colab/A5-CBOW/5_CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rohit Rawate

Assignment : 5

DL : CBOW

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as pylab
import numpy as np
%matplotlib inline

In [2]:
import re

In [3]:
sentences = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

Clean data

In [ ]:
# remove special characters
sentences = re.sub('[^A-Za-z0-9]+', ' ', sentences)

# remove 1 letter words
sentences = re.sub(r'(?:^| )\w(?:$| )', ' ', sentences).strip()

# lower all characters
sentences = sentences.lower()

Vocabulary/

In [7]:
words = sentences.split()
vocab = set(words)
vocab

{'As',
 'Computational',
 'In',
 'People',
 'The',
 'We',
 'a',
 'about',
 'abstract',
 'are',
 'beings',
 'by',
 'called',
 'computational',
 'computer',
 'computers.',
 'conjure',
 'create',
 'data.',
 'direct',
 'directed',
 'effect,',
 'evolution',
 'evolve,',
 'idea',
 'inhabit',
 'is',
 'manipulate',
 'of',
 'other',
 'our',
 'pattern',
 'process',
 'process.',
 'processes',
 'processes.',
 'program.',
 'programs',
 'rules',
 'spells.',
 'spirits',
 'study',
 'that',
 'the',
 'they',
 'things',
 'to',
 'we',
 'with'}

In [12]:
vocab_size = len(vocab) # 49
embed_dim = 10
context_size = 2
# vocab_size

Implementation 

In [13]:
word_to_ix = {word: i for i, word in enumerate(vocab)}  # word -> map  'idea' :  0
ix_to_word = {i: word for i, word in enumerate(vocab)}  # map -> word    0    : 'idea'
ix_to_word

{0: 'idea',
 1: 'processes.',
 2: 'process',
 3: 'People',
 4: 'As',
 5: 'In',
 6: 'We',
 7: 'The',
 8: 'program.',
 9: 'about',
 10: 'called',
 11: 'effect,',
 12: 'with',
 13: 'inhabit',
 14: 'manipulate',
 15: 'data.',
 16: 'by',
 17: 'processes',
 18: 'beings',
 19: 'our',
 20: 'rules',
 21: 'spirits',
 22: 'is',
 23: 'to',
 24: 'computers.',
 25: 'computer',
 26: 'spells.',
 27: 'that',
 28: 'the',
 29: 'are',
 30: 'we',
 31: 'Computational',
 32: 'evolve,',
 33: 'study',
 34: 'evolution',
 35: 'a',
 36: 'programs',
 37: 'direct',
 38: 'pattern',
 39: 'computational',
 40: 'things',
 41: 'directed',
 42: 'create',
 43: 'abstract',
 44: 'conjure',
 45: 'they',
 46: 'other',
 47: 'of',
 48: 'process.'}

**Data bags**

In [14]:
# data - [(context), target]

data = []
for i in range(2, len(words) - 2):
    context = [words[i - 2], words[i - 1], words[i + 1], words[i + 2]]
    target = words[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [15]:
embeddings =  np.random.random_sample((vocab_size, embed_dim))

Linear Model

In [16]:
def linear(m, theta):
    w = theta
    return m.dot(w)

*Log softmax + NLLloss = Cross Entropy*

In [17]:
def log_softmax(x):
    e_x = np.exp(x - np.max(x))
    return np.log(e_x / e_x.sum())

In [18]:
def NLLLoss(logs, targets):
    out = logs[range(len(targets)), targets]
    return -out.sum()/len(out)

In [19]:
def log_softmax_crossentropy_with_logits(logits,target):

    out = np.zeros_like(logits)
    out[np.arange(len(logits)),target] = 1
    
    softmax = np.exp(logits) / np.exp(logits).sum(axis=-1,keepdims=True)
    
    return (- out + softmax) / logits.shape[0]

Forward function

In [20]:
def forward(context_idxs, theta):
    m = embeddings[context_idxs].reshape(1, -1)
    n = linear(m, theta)
    o = log_softmax(n)
    
    return m, n, o

Backward function

In [21]:
def backward(preds, theta, target_idxs):
    m, n, o = preds
    
    dlog = log_softmax_crossentropy_with_logits(n, target_idxs)
    dw = m.T.dot(dlog)
    
    return dw

In [22]:
theta = np.random.uniform(-1, 1, (2 * context_size * embed_dim, vocab_size))
